### Quick Description
This notebook downloads every video subtitle listed in **download_descriptions.csv**, and creates a single output file with all sentences concatenated

In [1]:
import pandas as pd
import subprocess
import logging
import os
import yaml

from youtube_transcript_api import YouTubeTranscriptApi
from tqdm import tqdm

In [6]:
filepaths = yaml.load(open("/home/guilherme/Desktop/dissertation/config/filepaths.yaml"))

/home/guilherme-resende/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


In [ ]:
df = pd.read_csv(filepaths["00_youtube_download_descriptions"])
for i, row in tqdm(df.iterrows(), total=len(df)):
    video_id_and_channel = row["url"].split("v=")[1]
    video_id = video_id_and_channel.split("&")[0]

    try:
        if row["has-en-sub"]:
            transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
            subtitles = transcript_list.find_manually_created_transcript(["en"]).fetch()
        else:
            transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
            subtitles = transcript_list.find_generated_transcript(["en"]).fetch()

        subtitles = [sentence["text"] for sentence in subtitles]
        subtitles = " ".join(subtitles)

        with open(os.path.join(filepaths["01_youtube_raw"], video_id), 'w') as f:
            f.write(subtitles)
    except Exception as e:
        logging.error(f"Error while downloading the {i}-th video.")